<a href="https://colab.research.google.com/github/monesh1981/GEE_Projectmkb/blob/main/rivertemporallandsat8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import ee
import geemap
from IPython.display import display, HTML
from PIL import Image
!pip install ffmpeg-python
import os
import base64

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project="ee-moneshbathre")

# Create an interactive map
m = geemap.Map()
m.add_basemap("SATELLITE")
m




Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [10]:
# Get user-drawn ROI
roi = m.user_roi

if roi is None:
    print("Please draw a polygon on the map to set the ROI.")
else:
    # Function to mask clouds using the quality band
    def mask_clouds(image):
        qa = image.select('QA_PIXEL')
        cloud_mask = qa.bitwiseAnd(1 << 4).eq(0)  # Cloud bit (bit 3) should be 0 (no clouds)
        return image.updateMask(cloud_mask)



In [11]:
 # Load Landsat 8 collection, filter by date and ROI
collection = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
                  .filterBounds(roi)
                  .filterDate("2015-01-01", "2024-12-31")
                 .map(mask_clouds))

    # Generate Landsat timelapse
out_gif = 'landsat8_timelapse.gif'
out_mp4 = out_gif.replace('.gif', '.mp4')

# Delete existing MP4 to force update
if os.path.exists(out_mp4):
    os.remove(out_mp4)

# Generate Landsat timelapse

geemap.landsat_timelapse(
        roi=roi,
        out_gif=out_gif,
        start_year=2015,
        end_year=2024,
        start_date='01-01',
        end_date='12-31',
        bands=['NIR', 'SWIR1', 'Red'],  # Best for change detection
        title='Landsat 8 Temporal Changes (2015-2024)',
        progress_bar_color='blue',
        mp4=True,  # Generate MP4 as well
        frames_per_second=1,
      # Adjust speed
    )


  # Function to display MP4 in Colab
def display_video_mp4(video_path):
        with open(video_path, 'rb') as f:
            video_base64 = base64.b64encode(f.read()).decode()
        video_html = f'''
        <video width="600" controls>
            <source src="data:video/mp4;base64,{video_base64}" type="video/mp4">
        </video>
        '''
        return HTML(video_html)

    # Display the GIF
        if os.path.exists(out_gif):
            display(Image.open(out_gif))
        else:
            print(f"Error: The GIF file '{out_gif}' was not found.")

    # Display the MP4
if os.path.exists(out_mp4):
        display(display_video_mp4(out_mp4))
else:
        print(f"Error: The MP4 file '{out_mp4}' was not found.")





Generating URL...
Please wait ...
The GIF image has been saved to: /content/landsat8_timelapse.gif
